<a href="https://colab.research.google.com/github/ahmadmasood43/Glioblastoma-Detection-and-VR-Conversion/blob/main/segmentation_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Activation, MaxPooling2D, concatenate
from skimage.draw import polygon2mask, ellipse_perimeter, rectangle_perimeter, circle_perimeter

In [ ]:
from skimage.draw import polygon, circle_perimeter, ellipse_perimeter, rectangle_perimeter

# Read the CSV file
import pandas as pd
# Load the annotation CSV file
df = pd.read_csv('/content/gdrive/MyDrive/new_fyp_dataset_folder/annotation_fyp_all_yes.csv')

# Create lists to store image paths and corresponding masks
image_paths = []
masks = []

# Iterate over each row in the CSV file
for index, row in df.iterrows():
    # Read the image file
    filename = row['filename']
    image = imread(filename)

    # Extract region shape attributes
    region_shape = eval(row['region_shape_attributes'])
    shape_name = region_shape['name']
    shape_attributes = region_shape['attributes']

    if shape_name == 'polygon':
        # Extract polygon coordinates
        polygon_x = region_shape['all_points_x']
        polygon_y = region_shape['all_points_y']

        # Create a binary mask from the polygon coordinates
        mask = np.zeros_like(image[:, :, 0])
        mask[polygon_y, polygon_x] = 1
    elif shape_name == 'circle':
        # Extract circle attributes
        center_x = region_shape['cx']
        center_y = region_shape['cy']
        radius = region_shape['r']
        
        # Create a binary mask for the circle
        mask = np.zeros_like(image[:, :, 0])
        rr, cc = circle_perimeter(int(center_y), int(center_x), int(radius), shape=mask.shape)
        mask[rr, cc] = 1
    elif shape_name == 'ellipse':
        # Extract ellipse attributes
        center_x = region_shape['cx']
        center_y = region_shape['cy']
        radius_x = region_shape['rx']
        radius_y = region_shape['ry']
        rotation = region_shape['theta']
        
        # Create a binary mask for the ellipse
        mask = np.zeros_like(image[:, :, 0])
        rr, cc = ellipse_perimeter(int(center_y), int(center_x), int(radius_y), int(radius_x), orientation=np.deg2rad(rotation), shape=mask.shape)
        mask[rr, cc] = 1
    elif shape_name == 'rectangle':
        # Extract rectangle attributes
        x = region_shape['x']
        y = region_shape['y']
        width = region_shape['width']
        height = region_shape['height']
        
        # Create a binary mask for the rectangle
        mask = np.zeros_like(image[:, :, 0])
        rr, cc = rectangle_perimeter((int(y), int(x)), extent=(int(height), int(width)), shape=mask.shape)
        mask[rr, cc] = 1
    else:
        raise ValueError(f"Unsupported region shape: {shape_name}")




NameError: ignored

In [ ]:
# Define a custom data generator for loading images and masks
class DataGenerator(keras.utils.Sequence):
    def __init__(self, image_paths, masks, batch_size=32, image_size=(256, 256)):
        self.image_paths = image_paths
        self.masks = masks
        self.batch_size = batch_size
        self.image_size = image_size

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, idx):
        batch_image_paths = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_masks = self.masks[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_images = []
        for image_path in batch_image_paths:
            image = cv2.imread(image_path)
            image = cv2.resize(image, self.image_size)
            batch_images.append(image)

        batch_images = np.array(batch_images) / 255.0
        batch_masks = np.array(batch_masks) / 255.0
        batch_masks = np.expand_dims(batch_masks, axis=-1)

        return batch_images, batch_masks

# Create the U-Net model architecture
def unet(input_size=(256, 256, 3)):
    inputs = keras.Input(input_size)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bridge
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)

    # Decoder
    up5 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv4), conv3], axis=-1)
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(up5)
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv5), conv2], axis=-1)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv6), conv1], axis=-1)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(conv7)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv7)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the U-Net model
model = unet()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Create the training and validation data generators
train_generator = DataGenerator(image_paths[:800], masks[:800])
val_generator = DataGenerator(image_paths[800:], masks[800:])

# Train the model
model.fit(train_generator, epochs=10, validation_data=val_generator)

# Save the trained model
model.save('tumor_segmentation_model.h5')
